In [303]:
import sort_run.sort_run.constants as constants
cache_folder_name = 'simulation_study'
seed = 42

In [ ]:
import numpy as np
np.random.seed(seed)
import python_utils.python_utils.basic as basic
import python_utils.python_utils.caching as caching
log_folder = '%s/%s' % (constants.cache_folder, cache_folder_name)
caching.init(constants.cache_folder, constants.which_hash, basic.archiver(log_folder))
%load_ext autoreload
%autoreload 2

[Errno 17] File exists: '/Users/fultonw/Documents/projects/runs/sort_run/cache/simulation_study'
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
#plt.ion()
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import pandas as pd
import pdb
import itertools
import functools
import scipy.stats
import tensorflow as tf
import sort.sort.fxns as fxns

In [ ]:
# reset, start session
from tensorflow.python.framework import ops
ops.reset_default_graph()
sess = tf.Session()

In [ ]:
# define config
config = fxns.config()

In [ ]:
# define number of batches
num_batches = 10

In [ ]:
# generate xs_batches
xs_batches = [[np.random.normal(size=(config.batch_size, config.x_dim)) for j in xrange(config.num_steps)] for i in xrange(num_batches)]

In [ ]:
# generate ys by initializing a model, plugging in xs batches to get ps
model = fxns.basic_model(config)
init = tf.initialize_all_variables()
sess.run(init)
scores_batches = [None for i in xrange(num_batches)]
ps_batches = [None for i in xrange(num_batches)]
ys_batches = [None for i in xrange(num_batches)]
for (i, xs) in enumerate(xs_batches):
    scores_batches[i], ps_batches[i], = sess.run([model.score, model.p], feed_dict = {model.input_ph[i]:xs[i] for i in xrange(config.num_steps)})
    one_prob = (np.random.uniform(size=config.batch_size) < ps_batches[i][:,1]).astype(int)
    ys_batches[i] = np.array([1-one_prob,one_prob]).T

In [ ]:
# run some training epochs
num_epochs = 10
data_iterator = zip(xs_batches, ys_batches)
for j in xrange(num_epochs):
    model.run_epoch(sess, data_iterator)